# DECAGON Training

In [1]:
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations, chain
import time
import os
import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics
import pandas as pd
import psutil
import pickle
from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Train on GPU
#os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"] = '0'
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

In [ ]:
# psutil & time BEGIN
start = time.time() #in seconds
pid = os.getpid()
ps= psutil.Process(pid)

# Import Data from previous computations

In [4]:
filename = './data/data_structures/DECAGON_real_DSE_NPF'
with open(filename, 'rb') as f:
    DS = pickle.load(f)
    for key in DS.keys():
        globals()[key]=DS[key]
        print(key,"Imported successfully")

se_mono_name2idx Imported successfully
gene2idx Imported successfully
nonzero_feat Imported successfully
edge_type2dim Imported successfully
adj_mats_orig Imported successfully
edge_type2decoder Imported successfully
se_combo_name2idx Imported successfully
drug2idx Imported successfully
degrees Imported successfully
edge_types Imported successfully
num_edge_types Imported successfully
num_feat Imported successfully
feat Imported successfully


# Functions

In [9]:
def get_accuracy_scores(edges_pos, edges_neg, edge_type):
    feed_dict.update({placeholders['dropout']: 0})
    feed_dict.update({placeholders['batch_edge_type_idx']: minibatch.edge_type2idx[edge_type]})
    feed_dict.update({placeholders['batch_row_edge_type']: edge_type[0]})
    feed_dict.update({placeholders['batch_col_edge_type']: edge_type[1]})
    rec = sess.run(opt.predictions, feed_dict=feed_dict)

    def sigmoid(x):
        return 1. / (1 + np.exp(-x))

    # Predict on test set of edges
    preds = []
    actual = []
    predicted = []
    edge_ind = 0
    for u, v in edges_pos[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 1, 'Problem 1'

        actual.append(edge_ind)
        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_neg = []
    for u, v in edges_neg[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds_neg.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 0, 'Problem 0'

        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_all = np.hstack([preds, preds_neg])
    preds_all = np.nan_to_num(preds_all)
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    predicted = list(zip(*sorted(predicted, reverse=True, key=itemgetter(0))))[1]

    roc_sc = metrics.roc_auc_score(labels_all, preds_all)
    aupr_sc = metrics.average_precision_score(labels_all, preds_all)
    apk_sc = rank_metrics.apk(actual, predicted, k=50)

    return roc_sc, aupr_sc, apk_sc


def construct_placeholders(edge_types):
    placeholders = {
        'batch': tf.placeholder(tf.int32, name='batch'),
        'batch_edge_type_idx': tf.placeholder(tf.int32, shape=(), name='batch_edge_type_idx'),
        'batch_row_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_row_edge_type'),
        'batch_col_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_col_edge_type'),
        'degrees': tf.placeholder(tf.int32),
        'dropout': tf.placeholder_with_default(0., shape=()),
    }
    placeholders.update({
        'adj_mats_%d,%d,%d' % (i, j, k): tf.sparse_placeholder(tf.float32)
        for i, j in edge_types for k in range(edge_types[i,j])})
    placeholders.update({
        'feat_%d' % i: tf.sparse_placeholder(tf.float32)
        for i, _ in edge_types})
    return placeholders

## Settings and placeholders

In [10]:
val_test_size = 0.05
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_integer('neg_sample_size', 1, 'Negative sample size.')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 10, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0.1, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('max_margin', 0.1, 'Max margin parameter in hinge loss')
flags.DEFINE_integer('batch_size', 512, 'minibatch size.')
flags.DEFINE_boolean('bias', True, 'Bias term.')
# Important -- Do not evaluate/print validation performance every iteration as it can take
# substantial amount of time
PRINT_PROGRESS_EVERY = 150


In [11]:
print("Defining placeholders")
placeholders = construct_placeholders(edge_types)

Defining placeholders


In [12]:
# MACHETAZO!! Soluciona el bug de Jupyter con tensorflow que proporciona un flag -f
tf.app.flags.DEFINE_string('f', '', 'kernel')

## Create minibatch iterator, model and optimizer

In [13]:
print("Create minibatch iterator")
minibatch = EdgeMinibatchIterator(
    adj_mats=adj_mats_orig,
    feat=feat,
    edge_types=edge_types,
    batch_size=FLAGS.batch_size,
    val_test_size=val_test_size
)

Create minibatch iterator
Minibatch edge type: (0, 1, 0)
Constructing test edges= 0000/0914
Constructing val edges= 0000/0914


decagon/deep/minibatch.py:67: RuntimeWarning: divide by zero encountered in power
  rowdegree_mat_inv = sp.diags(np.nan_to_num(np.power(rowsum, -0.5)).flatten())
decagon/deep/minibatch.py:68: RuntimeWarning: divide by zero encountered in power
  coldegree_mat_inv = sp.diags(np.nan_to_num(np.power(colsum, -0.5)).flatten())


Train edges= 16463
Val edges= 0914
Test edges= 0914
Minibatch edge type: (1, 0, 0)
Constructing test edges= 0000/0914
Constructing val edges= 0000/0914
Train edges= 16463
Val edges= 0914
Test edges= 0914
Minibatch edge type: (0, 0, 0)
Constructing test edges= 0000/31122
Constructing test edges= 1000/31122
Constructing test edges= 2000/31122
Constructing test edges= 3000/31122
Constructing test edges= 4000/31122
Constructing test edges= 5000/31122
Constructing test edges= 6000/31122
Constructing test edges= 7000/31122
Constructing test edges= 8000/31122
Constructing test edges= 9000/31122
Constructing test edges= 10000/31122
Constructing test edges= 11000/31122
Constructing test edges= 12000/31122
Constructing test edges= 13000/31122
Constructing test edges= 14000/31122
Constructing test edges= 15000/31122
Constructing test edges= 16000/31122
Constructing test edges= 17000/31122
Constructing test edges= 18000/31122
Constructing test edges= 19000/31122
Constructing test edges= 20000/3112

Constructing val edges= 1000/2287
Constructing val edges= 2000/2287
Train edges= 41176
Val edges= 2287
Test edges= 2287
Minibatch edge type: (1, 1, 10)
Constructing test edges= 0000/2182
Constructing test edges= 1000/2182
Constructing test edges= 1000/2182
Constructing test edges= 2000/2182
Constructing val edges= 0000/2182
Constructing val edges= 1000/2182
Constructing val edges= 2000/2182
Train edges= 39288
Val edges= 2182
Test edges= 2182
Minibatch edge type: (1, 1, 11)
Constructing test edges= 0000/2169
Constructing test edges= 1000/2169
Constructing test edges= 2000/2169
Constructing val edges= 0000/2169
Constructing val edges= 1000/2169
Constructing val edges= 1000/2169
Constructing val edges= 1000/2169
Constructing val edges= 2000/2169
Train edges= 39044
Val edges= 2169
Test edges= 2169
Minibatch edge type: (1, 1, 12)
Constructing test edges= 0000/2163
Constructing test edges= 1000/2163
Constructing test edges= 2000/2163
Constructing val edges= 0000/2163
Constructing val edges= 

Constructing test edges= 1000/1584
Constructing val edges= 0000/1584
Constructing val edges= 1000/1584
Train edges= 28528
Val edges= 1584
Test edges= 1584
Minibatch edge type: (1, 1, 43)
Constructing test edges= 0000/1544
Constructing test edges= 1000/1544
Constructing val edges= 0000/1544
Constructing val edges= 1000/1544
Train edges= 27798
Val edges= 1544
Test edges= 1544
Minibatch edge type: (1, 1, 44)
Constructing test edges= 0000/1543
Constructing test edges= 1000/1543
Constructing val edges= 0000/1543
Constructing val edges= 1000/1543
Train edges= 27786
Val edges= 1543
Test edges= 1543
Minibatch edge type: (1, 1, 45)
Constructing test edges= 0000/1543
Constructing test edges= 1000/1543
Constructing val edges= 0000/1543
Constructing val edges= 1000/1543
Train edges= 27774
Val edges= 1543
Test edges= 1543
Minibatch edge type: (1, 1, 46)
Constructing test edges= 0000/1528
Constructing test edges= 1000/1528
Constructing val edges= 0000/1528
Constructing val edges= 1000/1528
Train edg

...

Constructing val edges= 0000/0154
Train edges= 2782
Val edges= 0154
Test edges= 0154
Minibatch edge type: (1, 1, 1635)
Constructing test edges= 0000/0153
Constructing val edges= 0000/0153
Train edges= 2766
Val edges= 0153
Test edges= 0153
Minibatch edge type: (1, 1, 1636)
Constructing test edges= 0000/0152
Constructing val edges= 0000/0152
Train edges= 2738
Val edges= 0152
Test edges= 0152
Minibatch edge type: (1, 1, 1637)
Constructing test edges= 0000/0151
Constructing val edges= 0000/0151
Train edges= 2732
Val edges= 0151
Test edges= 0151
Minibatch edge type: (1, 1, 1638)
Constructing test edges= 0000/0151
Constructing val edges= 0000/0151
Train edges= 2730
Val edges= 0151
Test edges= 0151
Minibatch edge type: (1, 1, 1639)
Constructing test edges= 0000/0151
Constructing val edges= 0000/0151
Train edges= 2726
Val edges= 0151
Test edges= 0151
Minibatch edge type: (1, 1, 1640)
Constructing test edges= 0000/0150
Constructing val edges= 0000/0150
Train edges= 2710
Val edges= 0150
Test edg

Constructing val edges= 0000/0119
Train edges= 2144
Val edges= 0119
Test edges= 0119
Minibatch edge type: (1, 1, 1696)
Constructing test edges= 0000/0119
Constructing val edges= 0000/0119
Train edges= 2142
Val edges= 0119
Test edges= 0119
Minibatch edge type: (1, 1, 1697)
Constructing test edges= 0000/0118
Constructing val edges= 0000/0118
Train edges= 2134
Val edges= 0118
Test edges= 0118
Minibatch edge type: (1, 1, 1698)
Constructing test edges= 0000/0117
Constructing val edges= 0000/0117
Train edges= 2112
Val edges= 0117
Test edges= 0117
Minibatch edge type: (1, 1, 1699)
Constructing test edges= 0000/0117
Constructing val edges= 0000/0117
Train edges= 2110
Val edges= 0117
Test edges= 0117
Minibatch edge type: (1, 1, 1700)
Constructing test edges= 0000/0117
Constructing val edges= 0000/0117
Train edges= 2108
Val edges= 0117
Test edges= 0117
Minibatch edge type: (1, 1, 1701)
Constructing test edges= 0000/0117
Constructing val edges= 0000/0117
Train edges= 2108
Val edges= 0117
Test edg

Constructing val edges= 0000/0098
Train edges= 1764
Val edges= 0098
Test edges= 0098
Minibatch edge type: (1, 1, 1750)
Constructing test edges= 0000/0097
Constructing val edges= 0000/0097
Train edges= 1762
Val edges= 0097
Test edges= 0097
Minibatch edge type: (1, 1, 1751)
Constructing test edges= 0000/0097
Constructing val edges= 0000/0097
Train edges= 1750
Val edges= 0097
Test edges= 0097
Minibatch edge type: (1, 1, 1752)
Constructing test edges= 0000/0097
Constructing val edges= 0000/0097
Train edges= 1750
Val edges= 0097
Test edges= 0097
Minibatch edge type: (1, 1, 1753)
Constructing test edges= 0000/0097
Constructing val edges= 0000/0097
Train edges= 1748
Val edges= 0097
Test edges= 0097
Minibatch edge type: (1, 1, 1754)
Constructing test edges= 0000/0097
Constructing val edges= 0000/0097
Train edges= 1746
Val edges= 0097
Test edges= 0097
Minibatch edge type: (1, 1, 1755)
Constructing test edges= 0000/0096
Constructing val edges= 0000/0096
Train edges= 1746
Val edges= 0096
Test edg

Train edges= 1408
Val edges= 0078
Test edges= 0078
Minibatch edge type: (1, 1, 1814)
Constructing test edges= 0000/0077
Constructing val edges= 0000/0077
Train edges= 1404
Val edges= 0077
Test edges= 0077
Minibatch edge type: (1, 1, 1815)
Constructing test edges= 0000/0077
Constructing val edges= 0000/0077
Train edges= 1400
Val edges= 0077
Test edges= 0077
Minibatch edge type: (1, 1, 1816)
Constructing test edges= 0000/0077
Constructing val edges= 0000/0077
Train edges= 1398
Val edges= 0077
Test edges= 0077
Minibatch edge type: (1, 1, 1817)
Constructing test edges= 0000/0077
Constructing val edges= 0000/0077
Train edges= 1396
Val edges= 0077
Test edges= 0077
Minibatch edge type: (1, 1, 1818)
Constructing test edges= 0000/0076
Constructing val edges= 0000/0076
Train edges= 1380
Val edges= 0076
Test edges= 0076
Minibatch edge type: (1, 1, 1819)
Constructing test edges= 0000/0076
Constructing val edges= 0000/0076
Train edges= 1378
Val edges= 0076
Test edges= 0076
Minibatch edge type: (1, 

Constructing val edges= 0000/0062
Train edges= 1134
Val edges= 0062
Test edges= 0062
Minibatch edge type: (1, 1, 1876)
Constructing test edges= 0000/0062
Constructing val edges= 0000/0062
Train edges= 1132
Val edges= 0062
Test edges= 0062
Minibatch edge type: (1, 1, 1877)
Constructing test edges= 0000/0062
Constructing val edges= 0000/0062
Train edges= 1124
Val edges= 0062
Test edges= 0062
Minibatch edge type: (1, 1, 1878)
Constructing test edges= 0000/0062
Constructing val edges= 0000/0062
Train edges= 1116
Val edges= 0062
Test edges= 0062
Minibatch edge type: (1, 1, 1879)
Constructing test edges= 0000/0061
Constructing val edges= 0000/0061
Train edges= 1116
Val edges= 0061
Test edges= 0061
Minibatch edge type: (1, 1, 1880)
Constructing test edges= 0000/0061
Constructing val edges= 0000/0061
Train edges= 1112
Val edges= 0061
Test edges= 0061
Minibatch edge type: (1, 1, 1881)
Constructing test edges= 0000/0061
Constructing val edges= 0000/0061
Train edges= 1102
Val edges= 0061
Test edg

In [14]:
print("Create model")
model = DecagonModel(
    placeholders=placeholders,
    num_feat=num_feat,
    nonzero_feat=nonzero_feat,
    edge_types=edge_types,
    decoders=edge_type2decoder,
)

Create model
Instructions for updating:
dim is deprecated, use axis instead


In [15]:
print("Create optimizer")
with tf.name_scope('optimizer'):
    opt = DecagonOptimizer(
        embeddings=model.embeddings,
        latent_inters=model.latent_inters,
        latent_varies=model.latent_varies,
        degrees=degrees,
        edge_types=edge_types,
        edge_type2dim=edge_type2dim,
        placeholders=placeholders,
        batch_size=FLAGS.batch_size,
        margin=FLAGS.max_margin
    )

Create optimizer


/home/juan/anaconda3/envs/py2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
print("Initialize session")
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed_dict = {}

Initialize session


# Train model

In [ ]:
acc_scores = np.array([None,None,None,None,None])
print("Train model")
for epoch in range(FLAGS.epochs):

    minibatch.shuffle()
    itr = 0
    while not minibatch.end():
        # Construct feed dictionary
        feed_dict = minibatch.next_minibatch_feed_dict(placeholders=placeholders)
        feed_dict = minibatch.update_feed_dict(
            feed_dict=feed_dict,
            dropout=FLAGS.dropout,
            placeholders=placeholders)

        t = time.time()

        # Training step: run single weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.batch_edge_type_idx], feed_dict=feed_dict)
        train_cost = outs[1]
        batch_edge_type = outs[2]

        if itr % PRINT_PROGRESS_EVERY == 0:
            val_auc, val_auprc, val_apk = get_accuracy_scores(
                minibatch.val_edges, minibatch.val_edges_false,
                minibatch.idx2edge_type[minibatch.current_edge_type_idx])
            step_time = time.time() - t
            
            print("Epoch:", "%04d" % (epoch + 1), "Iter:", "%04d" % (itr + 1), "Edge:", "%04d" % batch_edge_type,
                  "train_loss=", "{:.5f}".format(train_cost),
                  "val_roc=", "{:.5f}".format(val_auc), "val_auprc=", "{:.5f}".format(val_auprc),
                  "val_apk=", "{:.5f}".format(val_apk), "time=", "{:.5f}".format(step_time))
        itr += 1
        
acc_scores=acc_scores[1:,:]
output_data = {}
output_data['val_auc'] = acc_scores[:,0]
output_data['val_auprc'] = acc_scores[:,1]
output_data['val_apk'] = acc_scores[:,2]
output_data['train_cost'] = acc_scores[:,3]
output_data['step_time'] = acc_scores[:,4]
print("Optimization finished!")
for et in range(num_edge_types):
    roc_score, auprc_score, apk_score = get_accuracy_scores(
        minibatch.test_edges, minibatch.test_edges_false, minibatch.idx2edge_type[et])
    print("Edge type=", "[%02d, %02d, %02d]" % minibatch.idx2edge_type[et])
    print("Edge type:", "%04d" % et, "Test AUROC score", "{:.5f}".format(roc_score))
    print("Edge type:", "%04d" % et, "Test AUPRC score", "{:.5f}".format(auprc_score))
    print("Edge type:", "%04d" % et, "Test AP@k score", "{:.5f}".format(apk_score))
    print()
memUse = ps.memory_info()
print('Virtual memory:', memUse.vms)
print('RSS Memory:', memUse.rss)
total_time=time.time()-start
output_data['time'] = total_time
output_data['vms'] = memUse.vms
output_data['rss'] = memUse.rss
print("Total time:",total_time)
filename = 'results_training/TRAIN_real_DSE_NPF_epochs'+str(FLAGS.epochs)+'_h1'+str(FLAGS.hidden1)+\
           '_h2'+str(FLAGS.hidden2)+'_lr'+str(FLAGS.learning_rate)+'dropout'+str(FLAGS.dropout)
with open(filename, 'wb') as f:
    pickle.dump(output_data, f, protocol=2)